In [19]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

import matplotlib.pyplot as plt

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix

import prepare_readme as p
import explore as e

In [2]:
df = p.prep_readme()

In [3]:
df.head()

,repo,language,readme_contents,readme_contents_stemmed,readme_contents_lemmatized
0,/nakov/Practical-Cryptography-for-Developers-Book,CSS,# Welcome\n\n> **Warning**: this book is **not...,welcom warn book finish still work chapter com...,welcome warning book finished still working ch...
1,/eclipse/deeplearning4j,Java,# Monorepo of Deeplearning4j\n\nWelcome to the...,monorepo deeplearn j welcom new monorepo deepl...,monorepo deeplearning j welcome new monorepo d...
2,/h2oai/h2o-3,Java,# H2O\n\n[![Join the chat at https://gitter.im...,h join chat http gitter im h oai h http badg g...,h join chat http gitter im h oai h http badge ...
3,/tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www....",div align center img src http www tensorflow o...,div align center img src http www tensorflow o...
4,/grpc/grpc,C++,gRPC - An RPC library and framework\n=========...,grpc rpc librari framework grpc modern open so...,grpc rpc library framework grpc modern open so...


In [4]:
df_v = df.language.value_counts(dropna=False)

In [5]:
labels = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels

,n,percent
HTML,20,0.190476
C++,20,0.190476
Python,15,0.142857
CSS,10,0.095238
C,10,0.095238
JavaScript,10,0.095238
Shell,10,0.095238
Java,10,0.095238


In [20]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [7]:
# words that are programming languages only
#verterize ngrams

In [8]:
# restrict df to target and predicted veriables
df = df[['readme_contents_lemmatized','language']]

In [40]:
# vectorize stemmed readme contents and assign X any y veriables
tfidf = TfidfVectorizer(ngram_range=(1, 1),use_idf=False,sublinear_tf=True)
X = tfidf.fit_transform(df.readme_contents_lemmatized)
y = df.language

In [47]:
# # split data into test and train
# train, test = train_test_split(df)
# X_train = train.drop(columns=["language"])
# y_train = train.language
# X_test = test.drop(columns=["language"])
# y_test = test.language

,readme_contents_lemmatized,language
0,welcome warning book finished still working ch...,CSS
1,monorepo deeplearning j welcome new monorepo d...,Java
2,h join chat http gitter im h oai h http badge ...,Java
3,div align center img src http www tensorflow o...,C++
4,grpc rpc library framework grpc modern open so...,C++
...,...,...
100,usdk unity dsdk androidiossdknative c unitysdk...,HTML
101,circleci http circleci com gh eranif codelite ...,C++
102,quantlib free open source library quantitative...,C++
103,build status http travis ci org alehander pseu...,Python


In [11]:
def train_test_split(df):
    from sklearn.model_selection import train_test_split
    X = df.drop(columns=["language"])
    y = df.language
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8, random_state = 123, stratify=y)
    train = X_train.merge(y_train, left_index = True, right_index=True)
    test = X_test.merge(y_test, left_index=True, right_index=True)
    return train, test

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=.2)
X_train = pd.DataFrame(X_train.todense())
X_test = pd.DataFrame(X_test.todense())
train = X_train.copy()
train["language"]=y_train
test = X_test.copy()
test["language"]=y_test


In [60]:
# create classifier object and fit it to the data
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)
clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')


0         CSS
1        Java
2        Java
3         C++
4         C++
        ...  
100      HTML
101       C++
102       C++
103    Python
104    Python
Name: language, Length: 105, dtype: object

In [55]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.56


In [56]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 0.24


In [82]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
numeric_features = train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train.select_dtypes(include=['object']).drop(['language'], axis=1).columns
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),('cat', categorical_transformer, categorical_features)])
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)
model score: 0.976
model score: 0.381


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [145]:
classifiers = [
    #KNeighborsClassifier(12),
    #SVC(kernel="rbf", C=0.025, probability=True),
    #NuSVC(probability=True, random_state = 123),
    #DecisionTreeClassifier(min_samples_split=5, random_state=123),
    RandomForestClassifier(random_state=123, min_samples_split=10, n_estimators=1000,n_jobs=-1, oob_score=True),
    #AdaBoostClassifier(),
    #GradientBoostingClassifier()
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)
    print(classifier)
    print("model score: %.3f" % pipe.score(X_train, y_train))
    print("model score: %.3f" % pipe.score(X_test, y_test))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=True, random_state=123, verbose=0,
                       warm_start=False)
model score: 0.988
model score: 0.333


In [ ]:
def get_words(text):
    
    words = re.sub(r'[^\w\s]', '', text).split()
    return words

In [ ]:
def get_ngrams(n=2):

    ngram_master_list = []

    for row in df.readme_contents_stemmed.apply(get_words):

        bigrams = nltk.ngrams(row, n)

        ngram_master_list.extend(bigrams)
    
    return ngram_master_list

In [ ]:
def get_ngram_strings(ngrams):
    
    string_list = []

    for pair in ngrams:
    
        first = pair[0]

        second = pair[1]
    
        string_list.append(f"{first} {second}")
        
    return string_list

In [10]:
TfidfVectorizer()

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(2, 2))
X = tfidf.fit_transform()
y = df.language

In [ ]:
df['ngrams'] = e.get_ngrams(df,n=2)

In [ ]:
df.head(10)

In [ ]:
HTML_list =[]

for lst in df.readme_contents_stemmed.apply(p.tokenize)[df.language=='HTML']:
    
    HTML_list.extend(lst)
    
print(HTML_list)

In [ ]:
def tokenize(article):
    '''
    tokenizes words in a string
    '''

    # create token object
    tokenizer = nltk.tokenize.ToktokTokenizer()

    # use object to tokenize string
    article = tokenizer.tokenize(article, return_str=True)
    
    return article

#new svc or decition tree

In [ ]:
# create classifier object and fit it to the data
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)
clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')